In [10]:
!pip install tensorflow==1.12.0
!pip install coremltools

In [11]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input, Embedding, Flatten, concatenate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
import coremltools

# Model

In [0]:
# Swift for TensorFlow Model

# struct RegressionModel: Module {
#     var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
#     var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
#     var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
#     var embeddingLayer = Dense<Float>(inputSize: (4 + 45), outputSize: 64, activation: relu)
#     var allInputConcatLayer = Dense<Float>(inputSize: (32 + 64), outputSize: 128, activation: relu)
#     var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
#     var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
#     @differentiable
#     func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
#         let numericalInput = numericalLayer(input.numerical[0])
#         let embeddingOutput1 = embedding1(input.categorical[0])
#         let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
#             TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
#         let embeddingOutput2 = embedding2(input.categorical[1])
#         let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
#             TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
#         let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
#         let embeddingInput = embeddingLayer(embeddingConcat)
#         let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
#         return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
#     }
# }

In [0]:
numerical_input = Input(shape=(11,))
categorical_input1 = Input(shape=(2,))
categorical_input2 = Input(shape=(9,))

# numerical = Dense(32, activation='relu')(numerical_input)

embedding1 = Embedding(input_dim=2, output_dim=2, input_length=2)(categorical_input1)
embedding1 = Flatten()(embedding1)

embedding2 = Embedding(input_dim=9, output_dim=5, input_length=9)(categorical_input2)
embedding2 = Flatten()(embedding2)

# embeddingConcat = concatenate([embedding1, embedding2], axis=1)
# embedding = Dense(64, activation='relu')(embeddingConcat)

# allConcat = concatenate([numerical, embedding], axis=1)
allConcat = concatenate([numerical_input, embedding1, embedding2], axis=1)
hidden = Dense(128, activation='relu')(allConcat)
hidden = Dense(32, activation='relu')(hidden)
output = Dense(1)(hidden)

keras_model = Model(inputs=[numerical_input, categorical_input1, categorical_input2], outputs=output)

In [15]:
keras_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 9)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 2, 2)         4           input_5[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 9, 5)         45          input_6[0][0]                    
____________________________________________________________________________________________

# Save and Export no Train Model

In [0]:
keras_model.save('keras_simplified_no_train_model.h5')

In [17]:
mlmodel = coremltools.converters.keras.convert('keras_simplified_no_train_model.h5', use_float_arraytype = True)
mlmodel.save('coreml_simplified_no_train.mlmodel')

0 : input_5, <keras.engine.input_layer.InputLayer object at 0x7f21de4a8ac8>
1 : input_6, <keras.engine.input_layer.InputLayer object at 0x7f21de4a8b38>
2 : embedding_3, <keras.layers.embeddings.Embedding object at 0x7f21de4a8be0>
3 : embedding_4, <keras.layers.embeddings.Embedding object at 0x7f21de4a8ba8>
4 : input_4, <keras.engine.input_layer.InputLayer object at 0x7f21de4a8c50>
5 : flatten_3, <keras.layers.core.Flatten object at 0x7f21de4a8d30>
6 : flatten_4, <keras.layers.core.Flatten object at 0x7f21de4a8cc0>
7 : concatenate_2, <keras.layers.merge.Concatenate object at 0x7f21de4a8c88>
8 : dense_4, <keras.layers.core.Dense object at 0x7f21de4a8ef0>
9 : dense_4__activation__, <keras.layers.core.Activation object at 0x7f21de4a89e8>
10 : dense_5, <keras.layers.core.Dense object at 0x7f21de452240>
11 : dense_5__activation__, <keras.layers.core.Activation object at 0x7f21de4a8a20>
12 : dense_6, <keras.layers.core.Dense object at 0x7f21de452358>


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [18]:
spec = mlmodel.get_spec()
spec

specificationVersion: 1
description {
  input {
    name: "input1"
    type {
      multiArrayType {
        shape: 11
        dataType: FLOAT32
      }
    }
  }
  input {
    name: "input2"
    type {
      multiArrayType {
        shape: 1
        dataType: FLOAT32
      }
    }
  }
  input {
    name: "input3"
    type {
      multiArrayType {
        shape: 1
        dataType: FLOAT32
      }
    }
  }
  output {
    name: "output1"
    type {
      multiArrayType {
        shape: 1
        dataType: FLOAT32
      }
    }
  }
  metadata {
    userDefined {
      key: "coremltoolsVersion"
      value: "3.3"
    }
  }
}
neuralNetwork {
  layers {
    name: "embedding_3"
    input: "input2"
    output: "embedding_3_output"
    embedding {
      inputDim: 2
      outputChannels: 2
      weights {
        floatValue: -0.0013655312359333038
        floatValue: 0.042151663452386856
        floatValue: 0.03777125105261803
        floatValue: 0.03564964607357979
      }
    }
  }
  layers 